In [38]:
# Imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc # Garbage Collector
import os
import sys

# --- IMPORTS DEEP LEARNING ---
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, concatenate, Dropout
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_log_error
import re

In [39]:
data = pd.read_csv('data/processed_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3029664 entries, 0 to 3029663
Data columns (total 34 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   id                         int64  
 1   date                       object 
 2   store_nbr                  int64  
 3   family                     object 
 4   sales                      float64
 5   onpromotion                int64  
 6   is_train                   int64  
 7   city                       object 
 8   state                      object 
 9   type                       object 
 10  cluster                    int64  
 11  dcoilwtico                 float64
 12  is_holiday                 int64  
 13  day                        int64  
 14  month                      int64  
 15  year                       int64  
 16  dayofweek                  int64  
 17  weekofyear                 int64  
 18  is_weekend                 int64  
 19  is_payday                  int64  
 20  tr

In [40]:
# --- LOG-TRANSFORMATION DE LA CIBLE ---

# On applique la transformation seulement sur les données d'entraînement (is_train == 1)
# car la colonne 'sales' du jeu de test contient des np.nan.

data.loc[data['is_train'] == 1, 'sales'] = np.log1p(data.loc[data['is_train'] == 1, 'sales'])

In [41]:
# ==============================================================================
# ÉTAPE 1: PRÉPARATION FINALE POUR LE MODÈLE DEEP LEARNING
# ==============================================================================
print("Préparation finale des données pour MLP...")

# 2. Nettoyage des noms de colonnes (pour LightGBM, mais bonne pratique aussi pour Keras)
def clean_feature_names(df):
    new_cols = []
    for col in df.columns:
        col = re.sub(r'[\[\]<>]', '', col)
        col = re.sub(r'\s+', '_', col)
        col = re.sub(r'[,()"-]', '', col)
        new_cols.append(col)
    df.columns = new_cols
    return df
data = clean_feature_names(data)

# 3. Supprimer la colonne transactions originale qui a servi à créer les lags
if 'transactions' in data.columns:
    data.drop(columns=['transactions'], inplace=True)

# Définition des features
FEATURES = [col for col in data.columns if col not in ['id', 'sales', 'is_train', 'date']]

# Séparation des features en types pour le modèle DL
CATEGORICAL_FEATURES_OHE = ['is_holiday', 'dayofweek', 'is_weekend', 'is_payday'] # Simple OHE (ou binaire)
CATEGORICAL_FEATURES_EMBEDDING = ['store_nbr', 'family', 'city', 'state', 'type', 'cluster', 'month', 'weekofyear'] # Mieux adapté aux Embeddings
NUMERICAL_FEATURES = [col for col in FEATURES if col not in CATEGORICAL_FEATURES_OHE and col not in CATEGORICAL_FEATURES_EMBEDDING]

# --- Opérations de Prétraitement Finales sur Dataframe ---

# 4. Encodage Label (Obligatoire pour les Embeddings)
# Les colonnes d'Embedding DOIVENT commencer à 0.
label_encoders = {}
for col in CATEGORICAL_FEATURES_EMBEDDING:
    if col in data.columns:
        le = LabelEncoder()
        # Le fit sur l'ensemble complet (train + test) pour éviter les clés inconnues
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

# 5. Normalisation des Features Numériques (MinMaxScaler)
scaler = MinMaxScaler()
data[NUMERICAL_FEATURES] = scaler.fit_transform(data[NUMERICAL_FEATURES])

# 6. One-Hot Encoding pour les petites catégories
data = pd.get_dummies(data, columns=CATEGORICAL_FEATURES_OHE, drop_first=True)

# Mise à jour des FEATURES (si de nouvelles colonnes OHE ont été créées)
FEATURES = [col for col in data.columns if col not in ['id', 'sales', 'is_train', 'date']]

Préparation finale des données pour MLP...


In [42]:
# ==============================================================================
# ÉTAPE 2: SÉPARATION DES DONNÉES EN ENTRAÎNEMENT ET TEST
# ==============================================================================
print("Séparation des jeux d'entraînement et de test...")
train_df = data[data['is_train'] == 1].copy()
test_df = data[data['is_train'] == 0].copy()

Séparation des jeux d'entraînement et de test...


In [43]:
# ==============================================================================
# ÉTAPE 3 BIS: CRÉATION D'UN JEU DE VALIDATION
# ==============================================================================
print("Avant conversion, dtype date:", train_df['date'].dtype)

# Forcer 'date' en datetime
train_df['date'] = pd.to_datetime(train_df['date'])

print("Après conversion, dtype date:", train_df['date'].dtype)

print("Création du jeu de validation...")

# On utilise la colonne 'date' QUI EXISTE DÉJÀ DANS train_df
last_train_date = train_df['date'].max()
validation_start_date = last_train_date - pd.DateOffset(days=15)

# On crée le masque DIRECTEMENT à partir de train_df. Les longueurs correspondront parfaitement.
valid_indices = train_df[train_df['date'] >= validation_start_date].index
train_indices = train_df[train_df['date'] < validation_start_date].index

# Créer les jeux de données partiels
train_part_df = train_df.loc[train_indices]
valid_df = train_df.loc[valid_indices]

print(f"Jeu d'entraînement partiel : {train_part_df.shape[0]} lignes")
print(f"Jeu de validation : {valid_df.shape[0]} lignes")

Avant conversion, dtype date: object
Après conversion, dtype date: datetime64[ns]
Création du jeu de validation...
Jeu d'entraînement partiel : 2972640 lignes
Jeu de validation : 28512 lignes


In [44]:
# ==============================================================================
# ÉTAPE 4: DÉFINITION DES FEATURES (X) ET DE LA CIBLE (y)
# ==============================================================================

# Séparation des jeux de données après le nettoyage et l'encodage
train_df = data[data['is_train'] == 1].copy()
test_df = data[data['is_train'] == 0].copy()

# 1. Séparation Train Part/Validation
train_part_df = train_df.loc[train_indices].copy()
valid_df = train_df.loc[valid_indices].copy()

# Cible y (déjà log-transformée dans la nouvelle étape 1)
y_train_part = train_part_df['sales'].values
y_valid = valid_df['sales'].values

# Features X
X_train_part_num = train_part_df[NUMERICAL_FEATURES].values
X_valid_num = valid_df[NUMERICAL_FEATURES].values
X_test_num = test_df[NUMERICAL_FEATURES].values

# Features OHE/Embedding (Attention à l'ordre des colonnes, Keras est strict)
X_train_part_cat = train_part_df[[col for col in FEATURES if col not in NUMERICAL_FEATURES]].values
X_valid_cat = valid_df[[col for col in FEATURES if col not in NUMERICAL_FEATURES]].values
X_test_cat = test_df[[col for col in FEATURES if col not in NUMERICAL_FEATURES]].values

test_ids = test_df['id'].copy()

# Libérer de la mémoire
del train_df, test_df, data, train_part_df, valid_df
gc.collect()

7296

In [45]:
# Debug rapide des dtypes
print("X_train_part_num dtypes:", X_train_part_num.dtype)
print("X_train_part_cat dtypes:", X_train_part_cat.dtype)
print("y_train_part dtype:", y_train_part.dtype)

# Conversion en types compatibles Keras
X_train_part_num = X_train_part_num.astype("float32")
X_valid_num = X_valid_num.astype("float32")
X_test_num = X_test_num.astype("float32")

X_train_part_cat = X_train_part_cat.astype("int32")
X_valid_cat = X_valid_cat.astype("int32")
X_test_cat = X_test_cat.astype("int32")

y_train_part = y_train_part.astype("float32")
y_valid = y_valid.astype("float32")

X_train_part_num dtypes: float64
X_train_part_cat dtypes: object
y_train_part dtype: float64


In [46]:
# ==============================================================================
# ÉTAPE 5: ENTRAÎNEMENT DU MODÈLE MLP (Deep Learning)
# ==============================================================================
print("\nEntraînement du Modèle MLP Global...")

# --- 5.1: Définition de l'Architecture ---

def create_mlp_model(numerical_features, embedding_features, categorical_features_ohe, label_encoders):
    # Dictionnaire pour stocker les inputs Keras (une par feature)
    inputs = {}
    
    # 1. Inputs pour les Embeddings (features catégorielles avec bcp de classes)
    embedding_outputs = []
    for col in embedding_features:
        # Input: Entiers (Label Encoded)
        vocab_size = len(label_encoders[col].classes_)
        embed_dim = max(1, vocab_size // 4) # Taille de l'embedding: sqrt(vocab_size) ou vocab_size / 4
        
        input_layer = Input(shape=(1,), name=f'input_{col}')
        inputs[col] = input_layer
        
        # Couche d'Embedding
        embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim, name=f'embedding_{col}')(input_layer)
        flatten = Flatten()(embedding)
        embedding_outputs.append(flatten)
        
    # 2. Inputs pour les features OHE (déjà gérées par pd.get_dummies) et Numériques
    
    # Keras ne prend qu'un seul grand input pour les features numériques/OHE
    input_numeric_ohe = Input(shape=(X_train_part_num.shape[1] + (X_train_part_cat.shape[1] - len(embedding_features)),), 
                          name='input_num_ohe')
    inputs['num_ohe'] = input_numeric_ohe
    
    # 3. Concatenation
    if embedding_outputs:
        # Concaténer les Embeddings avec les features numériques/OHE
        all_features = concatenate(embedding_outputs + [input_numeric_ohe])
    else:
        all_features = input_numeric_ohe
        
    # 4. Couches Dense du MLP
    x = Dense(1024, activation='relu')(all_features)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    # Output: 1 neurone pour la vente (régression)
    output_layer = Dense(1, activation='linear', name='output_sales')(x)
    
    # Construction et compilation du modèle
    # Le modèle prend toutes les couches d'entrée définies
    model = Model(inputs=list(inputs.values()), outputs=output_layer)
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

# --- 5.2: Création et Entraînement ---

# Préparer les inputs pour Keras (un input par embedding + l'input num/ohe)
def prepare_keras_inputs(X_cat, X_num, embedding_features):
    # X_cat = [embeddings | autres cat/OHE]
    
    X_embed = X_cat[:, :len(embedding_features)]
    X_ohe = X_cat[:, len(embedding_features):]
    
    keras_inputs = []
    
    # 1. Inputs d'Embedding (séparés, 1 colonne à la fois)
    for i in range(len(embedding_features)):
        # Keras veut un array de shape (N, 1) pour l'input d'Embedding
        keras_inputs.append(X_embed[:, i].reshape(-1, 1))
        
    # 2. Input Numérique/OHE (fusionné)
    X_num_ohe = np.hstack([X_num, X_ohe])
    keras_inputs.append(X_num_ohe)
    
    return keras_inputs

# Création du modèle
mlp_model = create_mlp_model(NUMERICAL_FEATURES, CATEGORICAL_FEATURES_EMBEDDING, CATEGORICAL_FEATURES_OHE, label_encoders)
# mlp_model.summary() # Décommenter pour voir le résumé du modèle

# Préparation des inputs
X_train_keras = prepare_keras_inputs(X_train_part_cat, X_train_part_num, CATEGORICAL_FEATURES_EMBEDDING)
X_valid_keras = prepare_keras_inputs(X_valid_cat, X_valid_num, CATEGORICAL_FEATURES_EMBEDDING)

# Entraînement
history = mlp_model.fit(
    X_train_keras, y_train_part,
    validation_data=(X_valid_keras, y_valid),
    epochs=10, # Commence avec 50, ajuster en fonction du temps et de la performance
    batch_size=4096, # Taille de lot élevée pour accélérer le training
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)


Entraînement du Modèle MLP Global...
Epoch 1/10
726/726 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step - loss: 0.8300 - root_mean_squared_error: 0.9111 - val_loss: 0.3387 - val_root_mean_squared_error: 0.5820
Epoch 2/10
726/726 ━━━━━━━━━━━━━━━━━━━━ 34s 47ms/step - loss: 0.2997 - root_mean_squared_error: 0.5474 - val_loss: 0.3061 - val_root_mean_squared_error: 0.5533
Epoch 3/10
726/726 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - loss: 0.2469 - root_mean_squared_error: 0.4969 - val_loss: 0.3099 - val_root_mean_squared_error: 0.5567
Epoch 4/10
726/726 ━━━━━━━━━━━━━━━━━━━━ 34s 47ms/step - loss: 0.2227 - root_mean_squared_error: 0.4719 - val_loss: 0.3092 - val_root_mean_squared_error: 0.5560
Epoch 5/10
726/726 ━━━━━━━━━━━━━━━━━━━━ 34s 47ms/step - loss: 0.2095 - root_mean_squared_error: 0.4577 - val_loss: 0.3092 - val_root_mean_squared_error: 0.5561
Epoch 6/10
726/726 ━━━━━━━━━━━━━━━━━━━━ 34s 47ms/step - loss: 0.2006 - root_mean_squared_error: 0.4479 - val_loss: 0.2835 - val_root_mean_squared_error: 0.5324
Ep

In [47]:
# ==============================================================================
# ÉTAPE 6: PRÉDICTION ET CRÉATION DU FICHIER DE SOUMISSION
# ==============================================================================
print("\nGénération des prédictions sur le jeu de test...")

# Préparation de l'input de test
X_test_keras = prepare_keras_inputs(X_test_cat, X_test_num, CATEGORICAL_FEATURES_EMBEDDING)

# Prédiction (sortie log-transformée)
predictions_log = mlp_model.predict(X_test_keras).flatten()

# Inverse de la Log Transformation (np.expm1)
predictions = np.expm1(predictions_log)
predictions[predictions < 0] = 0

submission_df = pd.DataFrame({'id': test_ids, 'sales': predictions})
submission_df.to_csv('submission_mlp_deeplearning.csv', index=False)

print("\nFichier 'submission_mlp_deeplearning.csv' créé avec succès !")
print(submission_df.head())


Génération des prédictions sur le jeu de test...
891/891 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

Fichier 'submission_mlp_deeplearning.csv' créé avec succès !
           id     sales
1684  3000888  3.550138
1685  3002670  3.384698
1686  3004452  3.664725
1687  3006234  3.616055
1688  3008016  1.955912
